In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, LineString, Polygon

In [ ]:
# import crash data
crashes_raw = gpd.read_file('https://opendata.arcgis.com/datasets/70392a096a8e431381f1f692aaa06afd_24.geojson')

In [ ]:
# import anc data
ancs = gpd.read_file('https://opendata.arcgis.com/datasets/fcfbf29074e549d8aff9b9c708179291_1.geojson')

In [ ]:
# import all address points
address_points = gpd.read_file('https://opendata.arcgis.com/datasets/aa514416aaf74fdc94748f1e56e7cc8a_0.geojson')

In [ ]:
# import all 311 service requests in 2020
all311_2020 = gpd.read_file('https://opendata.arcgis.com/datasets/82b33f4833284e07997da71d1ca7b1ba_11.geojson')

In [ ]:
len(all311_2020)

In [ ]:
# limit to just traffic safety assessments 
tsa_2020 = all311_2020[all311_2020['SERVICECODEDESCRIPTION'] == 'Traffic Safety Investigation']

In [ ]:
len(tsa_2020)

In [ ]:
# import vision zero safety requests
vision_zero = gpd.read_file('https://opendata.arcgis.com/datasets/3f28bc3ad77f49079efee0ac05d8464c_0.geojson')

In [ ]:
# join crashes to data natively at ANC level and add year
anc_crashes = gpd.sjoin(crashes_raw, ancs, how="inner", op='within')
anc_crashes['YEAR'] = anc_crashes.apply(lambda x: x.REPORTDATE[:4], axis=1)
#Number of crashes thus far in 2020 by ANC
pd.DataFrame(anc_crashes.groupby(['YEAR', 'NAME']).size()).loc['2020']

In [135]:
# try to roll up address points to desired level
# roll up address points to census block 
census_blocks = address_points.dissolve(by='CENSUS_BLOCK', aggfunc='first')

In [150]:
census_blocks.head()

,geometry,OBJECTID_12,OBJECTID,SITE_ADDRESS_PK,ADDRESS_ID,ROADWAYSEGID,STATUS,SSL,TYPE_,ENTRANCETYPE,...,LONGITUDE,ACTIVE_RES_UNIT_COUNT,RES_TYPE,ACTIVE_RES_OCCUPANCY_COUNT,WARD_2002,WARD_2012,ANC_2002,ANC_2012,SMD_2002,SMD_2012
CENSUS_BLOCK,,,,,,,,,,,,,,,,,,,,,
000100 1000,"MULTIPOINT (-77.06808 38.91670, -77.06782 38.9...",840673,12693.0,261707,261707,10757.0,ACTIVE,2156 0801,ADDRESS,OFFICIAL,...,-77.059434,0.0,NON RESIDENTIAL,0.0,Ward 2,Ward 2,ANC 2E,ANC 2E,SMD 2E07,SMD 2E07
000100 1001,"MULTIPOINT (-77.06697 38.91462, -77.06693 38.9...",840671,12691.0,261694,261694,19124.0,ACTIVE,2154 0050,ADDRESS,OFFICIAL,...,-77.065732,0.0,RESIDENTIAL,1.0,Ward 2,Ward 2,ANC 2E,ANC 2E,SMD 2E07,SMD 2E02
000100 1002,"MULTIPOINT (-77.06277 38.91294, -77.06276 38.9...",844498,14693.0,273049,273049,34517.0,ACTIVE,1282 0144,ADDRESS,OFFICIAL,...,-77.061125,4.0,RESIDENTIAL,4.0,Ward 2,Ward 2,ANC 2E,ANC 2E,SMD 2E07,SMD 2E07
000100 1003,"MULTIPOINT (-77.06081 38.91208, -77.06081 38.9...",844492,14673.0,273012,273012,17073.0,ACTIVE,1282 0238,ADDRESS,OFFICIAL,...,-77.060193,0.0,RESIDENTIAL,1.0,Ward 2,Ward 2,ANC 2E,ANC 2E,SMD 2E07,SMD 2E07
000100 1004,"MULTIPOINT (-77.06407 38.91320, -77.06403 38.9...",844489,14657.0,272971,272971,8955.0,ACTIVE,1281 0020,ADDRESS,OFFICIAL,...,-77.063924,0.0,RESIDENTIAL,1.0,Ward 2,Ward 2,ANC 2E,ANC 2E,SMD 2E07,SMD 2E07


In [ ]:
# check geo type
census_blocks['geometry'].geom_type

In [136]:
len(census_blocks)

5339

In [137]:
# remove points so census blocks can be rolled up to polygons
census_blocks=census_blocks[census_blocks['geometry'].geom_type != 'Point']

In [138]:
# check how many fell out
len(census_blocks)

4961

In [139]:
# limit the census blocks dataset to census blocks that can become polygons
census_blocks = census_blocks[census_blocks['geometry'].apply(lambda x: len(list(x)) > 2)]

In [140]:
len(census_blocks)

4715

In [141]:
census_block_polygons=census_blocks.copy()

In [142]:
# convert to polygons
census_block_polygons['geometry'] = census_block_polygons.apply(lambda x: Polygon(x.geometry), axis=1)

In [143]:
len(census_block_polygons)

4715

In [144]:
len(crashes_raw)

237494

In [145]:
# roll up crashes by census block
crashes = gpd.sjoin(crashes_raw, census_block_polygons, how="left", op='within')

In [146]:
len(crashes)

237507

In [147]:
crashes['YEAR'] = crashes.apply(lambda x: x.REPORTDATE[:4], axis=1)

In [ ]:
#do a sanity check on number of crashes by year and ward
ward_year_rollup=pd.DataFrame(crashes.fillna(-1).groupby(['YEAR', 'WARD_right', 'WARD_left']).size())

In [ ]:
ward_year_rollup.to_excel('crashes_by_year_and_ward.xlsx')

In [148]:
# how many crashes in 2020 didn't join to a census block
crashes_null=crashes[(crashes['WARD_right'].isnull()) & (crashes['YEAR'] == '2020')]

In [149]:
len(crashes_null)

13373

In [ ]:
# check distance between address points objectid 846387/MARID 15323 and all of the crashes listed at that MARID
crash_sample = crashes_raw.loc[crashes_raw['MARID'] == 15232]

In [ ]:
addr_sample = address_points.loc[address_points['ADDRESS_ID'] == 15232]

In [ ]:
for point in crash_sample['geometry']:
    print(point.distance(addr_sample['geometry'].iloc[0]))

In [ ]:
address_points_buf = address_points.copy()
address_points_buf['geometry'] = address_points_buf.apply(lambda x: x.geometry.buffer(0.0003), axis=1)